## Artifical Neural Network

Importing libraries

In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Data Preprocessing

Importing our dataset

In [15]:
df=pd.read_csv(r"D:\COURSES\Python\Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [16]:
X=df.iloc[:, 3:-1].values
y=df.iloc[:,-1].values

In [17]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [18]:
print(y)

[1 0 1 ... 1 1 0]


Encoding categorical data

In [19]:
#label encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

In [21]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [22]:
#OneHot encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))


In [23]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into the Training & Test set

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

Feature Scaling

In [26]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

### Building the ANN

Initializing the ANN

In [28]:
ann=tf.keras.models.Sequential()

Adding the input layer and the first hidden layer

In [29]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding the second hidden layer

In [30]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding the output layer

In [32]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

### Training the ANN

Compiling the ANN

In [35]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Training the ANN on the Training set

In [36]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.6683 - accuracy: 0.6200
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5234 - accuracy: 0.8005
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4677 - accuracy: 0.8060
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4331 - accuracy: 0.8139
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8261
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3895 - accuracy: 0.8374
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3766 - accuracy: 0.8428
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3683 - accuracy: 0.8461
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3616 - accuracy: 0.8478
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3565 - accura

### Making the predictions and evaluating the model

Predicting the results of a single observation

In [43]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 30ms/step
[[False]]


Predicting the Test set results

In [45]:
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the confusion matrix

In [46]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1529   66]
 [ 259  146]]


0.8375